In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


In [3]:
# Load the data from CSV file
df = pd.read_csv("bankruptcy.csv")
# df.head()

# Check for NaN Values
nan_rows = df[df.isna().any(axis=1)]
if len(nan_rows) > 0:
    print(nan_rows)
else:
    print("No NaN values found.")

No NaN values found.


### We will split the data into two different groups. The first dataframe will contain all 95 variables while the second dataframe will contain the top 10 variables with the highest correlation to bankruptcy. 

The cell below is for all variables

In [10]:
# Split the data into training and testing sets for all variables
X = df.drop("Bankrupt?", axis=1)
y = df["Bankrupt?"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)


The cell below is for the top 10 variables

In [14]:
# Split the data into training and testing sets for top 10 variables based on correlation with bankruptcy

corr_matrix = df.corr()
num_features = 10
corr_with_bankrupt = corr_matrix["Bankrupt?"].abs().sort_values(ascending=False)
top_corr_features = corr_with_bankrupt[1:num_features+1].index.tolist()
df_top10 = df[top_corr_features]
X_top10 = df_top10
y_top10 = df["Bankrupt?"]
X_train_top10, X_test_top10, y_train_top10, y_test_top10 = train_test_split(X_top10, y_top10, test_size=0.2,random_state=42)

### Building a Neural Network Model

This code creates a neural network model using the Keras library with two layers. The first layer is a fully connected (Dense) layer with 64 neurons, which takes an input of shape X_train.shape[1] (the number of features in the training data) and uses the ReLU activation function. The second layer is another Dense layer with a single neuron and uses the sigmoid activation function, which is commonly used for binary classification problems.

In summary, the model has an input layer with the same number of neurons as the number of features in the data, one hidden layer with 64 neurons, and an output layer with a single neuron that outputs a probability between 0 and 1 representing the likelihood of the input belonging to the positive class (bankruptcy in this case).

In [15]:
# Build the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


# Visualize the model
# plot_model(model, to_file='model.png', show_shapes=True)

model_top10 = Sequential()
model_top10.add(Dense(64, input_dim=X_train_top10.shape[1], activation='relu'))
model_top10.add(Dense(32, activation='relu'))
model_top10.add(Dense(32, activation='relu'))
model_top10.add(Dense(32, activation='relu'))
model_top10.add(Dense(32, activation='relu'))
model_top10.add(Dense(1, activation='sigmoid'))


model.compile is a function in Keras that compiles the model with the chosen loss function, optimizer, and metrics. In this case, the loss function chosen is binary_crossentropy, which is commonly used for binary classification problems. The optimizer chosen is adam, which is a commonly used optimization algorithm for neural networks. Finally, metrics is set to accuracy, which will be used to evaluate the performance of the model during training and testing.

In [16]:
# Compile the model for all variables
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Compile the model for top 10 variables
model_top10.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


#### This line of code trains the neural network model on the training data with the following parameters:

X_train and y_train: the input features and corresponding target variable used for training the model.
epochs=50: the number of times the model is trained on the entire training dataset.
batch_size=32: the number of samples that will be used in each batch of training.
validation_data=(X_test, y_test): the validation data on which the model will be evaluated after each epoch of training. The validation data is used to monitor the performance of the model on data that it has not seen during training.
During training, the model tries to minimize the binary cross-entropy loss function using the Adam optimizer. The accuracy metric is also computed during training to evaluate the performance of the model on the training data.

In [17]:
# Train the model for all variables
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Train the model for top 10 variables

model_top10.fit(X_train_top10, y_train_top10, epochs=50, batch_size=32, validation_data=(X_test_top10, y_test_top10))

Epoch 1/50
171/171 [==============================] - 2s 3ms/step - loss: 2052810.8750 - accuracy: 0.9437 - val_loss: 1088040.8750 - val_accuracy: 0.9604
Epoch 2/50
171/171 [==============================] - 0s 1ms/step - loss: 620240.0000 - accuracy: 0.9463 - val_loss: 734833.0625 - val_accuracy: 0.9128
Epoch 3/50
171/171 [==============================] - 0s 2ms/step - loss: 474327.9688 - accuracy: 0.9443 - val_loss: 722236.9375 - val_accuracy: 0.9567
Epoch 4/50
171/171 [==============================] - 0s 1ms/step - loss: 328746.1250 - accuracy: 0.9467 - val_loss: 424546.8750 - val_accuracy: 0.9428
Epoch 5/50
171/171 [==============================] - 0s 2ms/step - loss: 240331.9375 - accuracy: 0.9468 - val_loss: 351104.8438 - val_accuracy: 0.9435
Epoch 6/50
171/171 [==============================] - 0s 1ms/step - loss: 216545.8906 - accuracy: 0.9439 - val_loss: 340141.2188 - val_accuracy: 0.9362
Epoch 7/50
171/171 [==============================] - 0s 1ms/step - loss: 179240.9688 

### Seeing the accuracy of our NN model(s)

In [18]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print("Accuracy for dataset with all variables:", accuracy)

loss_top10, accuracy_top10 = model_top10.evaluate(X_test_top10, y_test_top10)

print("Accuracy for dataset with top 10 variables:", accuracy_top10)

43/43 [==============================] - 0s 890us/step - loss: 66126.3203 - accuracy: 0.9619
Accuracy for dataset with all variables: 0.9618768095970154
43/43 [==============================] - 0s 901us/step - loss: 0.1005 - accuracy: 0.9655
Accuracy for dataset with top 10 variables: 0.9655424952507019


### Trying to do some visualisation of the data here


In [ ]:
# Create a correlation matrix
corr_matrix = df.corr()

# Generate a heatmap using seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, cmap='coolwarm')

# Show the plot
plt.show()

In [ ]:
#print(df.columns.to_list)

bankrupt_df = df[['Bankrupt?', ' Cash Flow to Sales']]
bankrupt_corr_matrix = bankrupt_df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(bankrupt_corr_matrix, cmap='coolwarm', annot = True)
plt.title('Correlation with Bankrupt?')
plt.show()




In [ ]:
# Find the top 10 features with highest absolute correlation values with Bankrupt? column
num_features = 10
corr_with_bankrupt = corr_matrix["Bankrupt?"].abs().sort_values(ascending=False)
top_corr_features = corr_with_bankrupt[1:num_features+1].index.tolist()

# Create a heatmap of top 10 features with highest absolute correlation values
sns.set(style="white")
plt.figure(figsize=(10,8))
sns.heatmap(df[top_corr_features].corr(), annot=True, cmap="YlGnBu")
plt.title("Top 10 features with highest absolute correlation values with Bankrupt and their respective correlations with one another", fontsize=12)
plt.show()

In [ ]:
for feature in top_corr_features:
    sns.violinplot(data=df[feature], orient = "h")
    plt.title(feature)
    plt.show()

In [ ]:
for feature in top_corr_features:
    sns.boxplot(data = df[feature], orient = "h")
    plt.title(feature)
    q1, q3 = df[feature].quantile([0.25, 0.75])
    iqr = q3 - q1

    # Identify the lower and upper bounds
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)

    # Count the number of outliers
    outliers = len(df[(df[feature] < lower_bound) | (df[feature] > upper_bound)])

    print(f"Number of Outliers for{feature}:", outliers)

    plt.show()

In [ ]:
# Create scatter plot for each combination of top features
# for i in range(len(top_corr_features)):
#     for j in range(i+1, len(top_corr_features)):
#         feature1 = top_corr_features[i]
#         feature2 = top_corr_features[j]
#         sns.jointplot(x=feature1, y=feature2, data=df, kind='reg')
#         plt.show()

### Building a Decision Tree Model

This code trains a decision tree classifier model dectree with a maximum depth of 2 on the training data X_train and y_train.

After training, the model is visualized as a tree using the plot_tree function from sklearn.tree. The plot_tree function generates a tree diagram that shows the decision-making process of the decision tree model.

The filled=True argument fills the tree nodes with colors to indicate the class distribution, while the rounded=True argument makes the edges of the boxes around the nodes rounded.

The feature_names argument specifies the names of the input features or predictors, which are columns in the DataFrame X.

The class_names argument specifies the names of the target classes or outputs, which in this case are "Not Bankrupt" and "Bankrupt".

The resulting tree diagram can be used to interpret how the model makes predictions for new data based on the values of the input features.

In [ ]:
dectree = DecisionTreeClassifier(max_depth=2)
dectree.fit(X_train, y_train)

f = plt.figure(figsize=(24,24))
plot_tree(dectree, filled=True, rounded=True, 
          feature_names=X.columns, 
          class_names=["Not Bankrupt","Bankrupt"])

### Seeing the accuracy of our Decision Tree Model

In [ ]:
# Evaluate the accuracy of the model
y_pred = dectree.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the model: {:.2f}%".format(accuracy*100))
